In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
#EuroTop  2018 6.9


#Hydrodynamics

Hs = 0.11
Hs_range = np.linspace(0.6*Hs, 1.2*Hs, 5)
print(Hs_range)

#Gemometry

crest_height = 0.56
berm_height = 0.4
water_levels = np.array([0, 0.05])

